In [1]:
from pyspark import SparkContext, SQLContext

sc = SparkContext("local[*]")

In [18]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [2]:
sqlc = SQLContext(sc)

# USDA Nutrient Data

## Fetch raw data from official site

In [9]:
! cd data && wget https://www.ars.usda.gov/ARSUserFiles/80400525/Data/SR/SR28/dnload/sr28abbr.zip && unzip sr28abbr.zip 

--2018-05-19 14:50:21--  https://www.ars.usda.gov/ARSUserFiles/80400525/Data/SR/SR28/dnload/sr28abbr.zip
Resolving www.ars.usda.gov (www.ars.usda.gov)... 199.133.10.189
Connecting to www.ars.usda.gov (www.ars.usda.gov)|199.133.10.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1265639 (1.2M) [application/x-zip-compressed]
Saving to: ‘sr28abbr.zip.1’

sr28abbr.zip.1      100%[===================>]   1.21M  1.83MB/s    in 0.7s    

2018-05-19 14:50:37 (1.83 MB/s) - ‘sr28abbr.zip.1’ saved [1265639/1265639]

Archive:  sr28abbr.zip
  inflating: ABBREV.txt              
  inflating: sr28_doc.pdf            


## Ingest and split raw text file

In [10]:
raw_nutr = sc.textFile("data/ABBREV.txt")

In [12]:
raw_nutr.first()

'~01001~^~BUTTER,WITH SALT~^15.87^717^0.85^81.11^2.11^0.06^0.0^0.06^24^0.02^2^24^24^643^0.09^0.000^0.000^1.0^0.0^0.005^0.034^0.042^0.110^0.003^3^0^3^3^18.8^0.17^2499^684^671^0^158^0^0^0^2.32^0.0^0^7.0^51.368^21.021^3.043^215^5.0^~1 pat,  (1" sq, 1/3" high)~^14.2^~1 tbsp~^0'

In [13]:
split_nutr = raw_nutr.map(lambda x: x.split("^"))

In [20]:
first_row = split_nutr.first()

In [21]:
print(first_row)

['~01001~', '~BUTTER,WITH SALT~', '15.87', '717', '0.85', '81.11', '2.11', '0.06', '0.0', '0.06', '24', '0.02', '2', '24', '24', '643', '0.09', '0.000', '0.000', '1.0', '0.0', '0.005', '0.034', '0.042', '0.110', '0.003', '3', '0', '3', '3', '18.8', '0.17', '2499', '684', '671', '0', '158', '0', '0', '0', '2.32', '0.0', '0', '7.0', '51.368', '21.021', '3.043', '215', '5.0', '~1 pat,  (1" sq, 1/3" high)~', '14.2', '~1 tbsp~', '0']


## Generate schema to create DataFrame

In [ ]:
col_names = ["id", "description", ]

In [ ]:
def get_type(col):
    if "~" in col:
        return StringType()
    else:
        return IntegerType()

In [16]:
schema = StructType([
    StructField(col_names[i], get_type(first_row[i]), True)
    for i in range(len(first_row))
])

In [19]:
sqlc.createDataFrame?

In [ ]:
sqlc.createDataFrame(split_nutr, schema=)